In [1]:
# Step 1 Read in data
# Step 2 Preprocess text data
# Step 3 Word Embedding
# Step 4 LDA

In [25]:
pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis

SyntaxError: invalid syntax (<ipython-input-25-f6401eb890a1>, line 1)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re

In [2]:
yelp=pd.read_csv('all_data20180608.csv')

In [3]:
# PreProcessing
#step 1 lower case
#step 2 punctuation
#step 3 stop word
#step 4 common word removal
#step 5 rare word removal
#step 6 token
#step 7 stemming
#step 8 lemma

In [4]:
yelp['lower'] = yelp.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
yelp.lower.head()

0    my friend gabi, i love your cute parisian inte...
1     had a good waiter, all the staff were very cool.
2    my only regret is not catching the name of our...
3    lotus of siam did not disappoint, the service ...
4    his name is carlos if you ever want to request...
Name: lower, dtype: object

In [5]:
from nltk.tokenize import RegexpTokenizer
reg_tok = RegexpTokenizer(r'\w+')#+ is one or more
yelp['no_punc'] = yelp['lower'].apply(lambda x: ' '.join(reg_tok.tokenize(x)))
yelp.no_punc.head()

0    my friend gabi i love your cute parisian inter...
1       had a good waiter all the staff were very cool
2    my only regret is not catching the name of our...
3    lotus of siam did not disappoint the service w...
4    his name is carlos if you ever want to request...
Name: no_punc, dtype: object

In [6]:
yelp['no_stop'] = yelp['no_punc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
yelp.no_stop.head()

0    friend gabi love cute parisian interior dim li...
1                               good waiter staff cool
2    regret catching name server best experienced f...
3        lotus siam disappoint service great attentive
4          name carlos ever want request service great
Name: no_stop, dtype: object

In [7]:
freq = pd.Series(' '.join(yelp['no_stop']).split()).value_counts()[:20]#combining all rows and then splitting and converitign and value count
freq
#looking at these, we actually want to keep them so no need to carry out this step

food          3504
good          1925
buffet        1556
service       1554
great         1365
place         1111
vegas          882
like           764
restaurant     656
one            642
get            641
best           635
really         625
quality        611
price          596
would          552
time           539
go             539
selection      470
better         463
dtype: int64

In [8]:
rare = pd.Series(' '.join(yelp['no_stop']).split()).value_counts()[-600:]
rare


plunk            1
mixing           1
cruz             1
wold             1
gruelingly       1
unagi            1
miraculously     1
fam              1
expresso         1
capulet          1
unwilling        1
posters          1
streamlined      1
retter           1
slathered        1
ons              1
rachel           1
mascarpone       1
showcase         1
banned           1
prob             1
dismantled       1
resign           1
occurrence       1
con              1
tack             1
themes           1
dispute          1
reduce           1
slushly          1
                ..
gargle           1
highway          1
shopper          1
nicole           1
circumstances    1
7am              1
device           1
prima            1
middling         1
surrounded       1
scooping         1
overpowered      1
cus              1
swallowing       1
mondays          1
borgata          1
feathers         1
missy            1
lemons           1
unfortuantely    1
vul              1
unhealthy   

In [9]:
rare = list(rare.index)
yelp['no_rare'] = yelp['no_stop'].apply(lambda x: " ".join(x for x in x.split() if x not in rare))


In [10]:
from textblob import TextBlob
# not really doing that for tutorial, this is just demo of it
yelp['no_stop'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    friend gave love cut parisian interior dim lig...
1                               good waiter staff cool
2    regret catching name server best experienced f...
3         lots siam disappoint service great attentive
4           name carlo ever want request service great
Name: no_stop, dtype: object

In [11]:
from nltk.tokenize.treebank import TreebankWordTokenizer
_word_tokenize = TreebankWordTokenizer()
yelp['token'] = yelp['no_rare'].apply(lambda x: ' '.join(_word_tokenize.tokenize(x)))
yelp.token.head()

0    friend gabi love cute parisian interior dim li...
1                               good waiter staff cool
2    regret catching name server best experienced f...
3        lotus siam disappoint service great attentive
4          name carlos ever want request service great
Name: token, dtype: object

In [12]:
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer("english")
yelp['stemed']=yelp['token'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
yelp.stemed.head()

0    friend gabi love cute parisian interior dim li...
1                               good waiter staff cool
2    regret catch name server best experienc far tr...
3            lotus siam disappoint servic great attent
4            name carlo ever want request servic great
Name: stemed, dtype: object

In [13]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
yelp['lemma']=yelp['stemed'].apply(lambda x: " ".join([wordnet_lemmatizer.lemmatize(word) for word in x.split()]))
yelp.lemma.head(20)

0     friend gabi love cute parisian interior dim li...
1                                good waiter staff cool
2     regret catch name server best experienc far tr...
3             lotus siam disappoint servic great attent
4             name carlo ever want request servic great
5                               room beauti server good
6     servic quick price ok get pretti darn good san...
7                                 good servic good food
8     say locat decor lotus siam never life find bet...
9                              servic snappi food tasti
10    came month ago food ok initi encount cashier g...
11                       hostess waitress friend attent
12                     shout boy wesley host cool peopl
13                            waitress awesom help ball
14     servic great busi afternoon outdoor set look day
15    arriv 3pm weekday prompt seat busi patio time ...
16    happi help take mani pictur request alway kept...
17    item order mon ami gabi oyster du jour 15 

In [14]:
yelp.lemma.str.len().max()

617

In [15]:
from collections import Counter

count=Counter(" ".join(yelp.lemma).split(" ")).items()
print(sorted(count))

[('', 3), ('0', 10), ('00', 33), ('000', 6), ('00am', 1), ('00pm', 2), ('05', 1), ('06', 3), ('07', 2), ('1', 98), ('10', 95), ('100', 22), ('1000', 8), ('102', 2), ('10lbs', 1), ('10min', 1), ('10x', 1), ('11', 21), ('110', 5), ('112', 1), ('11am', 4), ('11pm', 1), ('12', 23), ('120', 1), ('12pm', 1), ('13', 23), ('14', 10), ('140', 3), ('15', 79), ('150', 1), ('1500', 1), ('15pm', 2), ('16', 13), ('164', 1), ('17', 9), ('1762', 2), ('18', 22), ('19', 7), ('1am', 1), ('1st', 7), ('2', 186), ('20', 69), ('200', 9), ('2002', 2), ('2008', 1), ('2010', 3), ('2011', 1), ('2012', 1), ('2016', 2), ('20min', 4), ('21', 7), ('22', 19), ('23', 2), ('23th', 1), ('24', 45), ('24hrs', 6), ('25', 42), ('26', 1), ('27', 13), ('270', 4), ('28', 4), ('29', 8), ('2nd', 14), ('2nds', 2), ('2pm', 3), ('3', 182), ('30', 83), ('300', 6), ('30am', 2), ('30ish', 2), ('30pm', 6), ('33', 5), ('330', 2), ('330pm', 1), ('34', 5), ('35', 28), ('35min', 1), ('36', 3), ('37', 5), ('38', 14), ('39', 2), ('3pm', 8), 

In [16]:
len(count)


5276

In [17]:
print(max(yelp.lemma, key=len))

like singl littl dish put tast portion deep fri broccoli chees casserol surpris favorit american plate love littl bucket tater tot waffl fri mini fri basket piec fri chicken sweet potato fri brisket nice rub outsid like option bbq sauc red velvet whoopi pie soft point flavor authent rendit dessert varieti cupcak cooki bread pud uniqu gelato flavor made order crepe sugar free dessert ton choos midlight good amount empti spot item look good guess popular ran lowlight shrimp cold one tast bit fishi hot one head overlook spici fri fish excit dish great probabl sit meat inconsist dri item great buffet other mediocr


In [ ]:
# print(model)

In [ ]:
# from sklearn.decomposition import PCA
# from matplotlib import pyplot
# X = model[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)
# # create a scatter plot of the projection
# pyplot.scatter(result[:, 0], result[:, 1])
# words = list(model.wv.vocab)
# for i, word in enumerate(words):
# 	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
# pyplot.show()

In [27]:
from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass

ModuleNotFoundError: No module named 'lda2vec'

In [ ]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2   

In [ ]:
# import tensorflow as tf
# import numpy as np
# valid_size = 16     # Random set of words to evaluate similarity on.
# valid_window = 100  # Only pick dev samples in the head of the distribution.
# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
# train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
# valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [ ]:
# embeddings = tf.Variable(
#     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
# embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [24]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
# define documents

# define class labels
encoder = LabelEncoder()
encoder.fit(yelp.category)
encoded_Y = encoder.transform(yelp.category)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
dictionary=Dictionary([list(yelp.lemma)])
# integer encode the documents
vocab_size = 5248
#encoded_docs = [one_hot(d, vocab_size) for d in yelp.lemma]
doc_term_matrix = [dictionary.doc2bow(doc) for doc in yelp.lemma]
print(doc_term_matrix)
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 102
#padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


#X_train, X_test, y_train, y_test = train_test_split(padded_docs,dummy_y,test_size=0.1)

Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=12, id2word = dictionary, passes=50)

# evaluate the model
#y_pred=model.predict(X_test)
print(ldamodel.print_topics(num_topics=3, num_words=3))

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
# from sklearn.metrics import confusion_matrix
# conf_mat = confusion_matrix(y_test, y_pred)
# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(conf_mat, annot=True, fmt='d',
#             xticklabels=category_id_df.category.values, yticklabels=category_id_df.category.values)
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.show()
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss))

In [ ]:
from sklearn import metrics
#print(metrics.classification_report(y_test, y_pred, target_names=my_df['category'].unique()))
print(metrics.accuracy_score(X_test, y_test))

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=12, id2word = dictionary, passes=50)

In [ ]:
print(y_test[0])

In [ ]:
# from lda2vec import LDA2Vec
# n_words = 10
# n_docs = 15
# n_hidden = 8
# n_topics = 2
# n_obs = 300
# words = np.random.randint(n_words, size=(n_obs))
# _, counts = np.unique(words, return_counts=True)
# model = LDA2Vec(n_words, n_hidden, counts)
# model.add_categorical_feature(n_docs, n_topics, name='document id')
# model.finalize()
# doc_ids = np.arange(n_obs) % n_docs
# loss = model.fit_partial(words, 1.0, categorical_features=doc_ids)